
<a href="https://colab.research.google.com/github/kokchun/Databehandling-AI22/blob/main/Lectures/L3-merge.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code

---
# Lecture notes - Merge, join, concatenate, append, read_html
---

This is the lecture note for **data wrangling** - but it's built upon contents from pandas and previous course: 
- Python programming

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to merging. I encourage you to read further about merging.

Read more 
- [Merge, join, concatenate ...](https://pandas.pydata.org/docs/user_guide/merging.html)

- [read_html()](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html?highlight=read_html#pandas.read_html)

- [append()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html)

---

## Setup
- creating 3 DataFrames

In [3]:
import pandas as pd 
import numpy as np 

df1 = pd.DataFrame(np.arange(16).reshape(4,4), columns=list("ABCD"))
df2 = pd.DataFrame(np.zeros((3,4)), columns=list("ABCD"))
df3 = pd.DataFrame(np.ones((4,3)), index=[1,5,10,15])
df1

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [4]:
df2

,A,B,C,D
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [5]:
df3

,0,1,2
1,1.0,1.0,1.0
5,1.0,1.0,1.0
10,1.0,1.0,1.0
15,1.0,1.0,1.0


---
## Concat

- concatenates pandas objects along particular axis
- there are optional set logic along other axes
- concat() copies the data, if using concat() on many datasets, use list comprehensions

Set logic: 

<img src="../assets/outer-inner-join.png" width=500/>

left side: $A\cup B$ (union)

right side: $A\cap B$ (intersection)

In [6]:
# same column names, just add the second df rows to the first
pd.concat([df1,df2]) # by default axis = 0 or "rows"

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [7]:
# concats on the index 
# NOTE: NaNs due to df2 shape is (3,4) while df1 shape is (4,4)
pd.concat([df1,df2], axis = "columns")

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0
3,12,13,14,15,NaN,NaN,NaN,NaN


In [8]:
# NOTE: index 1 is shared while others are not 
pd.concat([df1,df3], axis = 1)  # join="outer by default"

,A,B,C,D,0,1,2
0,0.0,1.0,2.0,3.0,NaN,NaN,NaN
1,4.0,5.0,6.0,7.0,1.0,1.0,1.0
2,8.0,9.0,10.0,11.0,NaN,NaN,NaN
3,12.0,13.0,14.0,15.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,1.0,1.0
10,NaN,NaN,NaN,NaN,1.0,1.0,1.0
15,NaN,NaN,NaN,NaN,1.0,1.0,1.0


In [9]:
# inner join - set logic intersection "A and B"
pd.concat([df1,df3], axis = 1, join="inner") 

,A,B,C,D,0,1,2
1,4,5,6,7,1.0,1.0,1.0


---
## Append

NOTE: append will become deprecated - use concat instead

Specific case of concat
- concatenates along axis 0 
- join = outer
- can't choose other axis or joins
- doesn't modify original df unlike in append for lists

In [10]:
df1.append(df2, ignore_index=True) # simple 

AttributeError: 'DataFrame' object has no attribute 'append'

---
## Merge

- high performance join operations similar to relational databases
- performance is faster using merge/join in Pandas than in SQL

Relational algebra
- one-to-one - joining 2 dfs on their index
- many-to-one - joining a unique index to ≥ 1 cols in different df
- many-to-many - joining columns on columns

In [29]:
# merge
left = pd.DataFrame({"key": ["K0", "K0", "K1", "K2"],
                     "A": ["A0", "A1", "A2", "A3"],
                     "B": ["B0", "B1", "B2", "B3"],
                     })
right = pd.DataFrame({"key": ["K0", "K1", "K2", "K3"],
                     "C": ["C0", "C1", "C2", "C3"],
                      "D": ["D0", "D1", "D2", "D3"]})
print(left)
print(right)


  key   A   B
0  K0  A0  B0
1  K0  A1  B1
2  K1  A2  B2
3  K2  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3


In [30]:
# note on column must be found in both left and right
pd.merge(left, right, on="key", indicator=True) # how= "inner" default


,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [31]:
# note that for key K3 there is only the right one and there is NaN in col A,B because 
# outer join is a union between left and right 
left.merge(right, on=["key"], how="outer", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both
4,K3,NaN,NaN,C3,D3,right_only


In [35]:
left.merge(right, on="key", how="left", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [38]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "A": ["A0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

print(left)

print(right)

# A_x is from left, A_y is from right, since A exist in both dataframes
# only keys present in left
left.merge(right, on=["key1", "key2"], how="left", indicator=True)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
  key1 key2   A   D
0   K0   K0  A0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3


,key1,key2,A_x,B,A_y,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K0,K1,A1,B1,NaN,NaN,left_only
2,K1,K0,A2,B2,C1,D1,both
3,K1,K0,A2,B2,C2,D2,both
4,K2,K1,A3,B3,NaN,NaN,left_only


In [43]:
# keys in right
left.merge(right, on=["key1", "key2"], how="right", indicator=True, suffixes=["_left","_right"])

,key1,key2,A_left,B,A_right,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K1,K0,A2,B2,C1,D1,both
2,K1,K0,A2,B2,C2,D2,both
3,K2,K0,NaN,NaN,C3,D3,right_only


---

## Join
- combine cols of two potentially different index dfs
- uses merge internally

In [46]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)


right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)
left, right


(     A   B
 K0  A0  B0
 K1  A1  B1
 K2  A2  B2,
      C   D
 K0  C0  D0
 K2  C2  D2
 K3  C3  D3)

In [48]:
left.join(right) # joins on left

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [50]:
right.join(left) # joins on right

,C,D,A,B
K0,C0,D0,A0,B0
K2,C2,D2,A2,B2
K3,C3,D3,NaN,NaN


In [51]:
left.join(right, how = "right")

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


---
## Webscrape HTML tables 

- [solve SSLCertVerificationError on mac](https://stackoverflow.com/questions/68275857/urllib-error-urlerror-urlopen-error-ssl-certificate-verify-failed-certifica)




In [20]:
tables_list = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_potentially_habitable_exoplanets")

tables_list.__len__()

8

In [22]:
df = tables_list[1] # first table in the html page 
df.head()

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/Notes
0,Earth (reported for reference),Sun,G2V,1.00,1.00,5.514,1.00,255,365.25,0,Only planet known to support life.
1,Venus (reported for reference),Sun,G2V,0.815,0.9499,5.243,1.911,244.261,224.7,4.2×10-6,[5]
2,Mars (reported for reference),Sun,G2V,0.107,0.532,3.934,0.431,209.8,686.976,5.77×10-6−4.24×10-5,[6]
3,Gliese 12 b,Gliese 12,M4V,0.88+0.39 −0.26,1.03±0.11,4.44,1.6±0.2,315,12.76144±0.00006,40,[7]
4,Gliese 163 c,Gliese 163,M3V,≥6.80,—,—,1.25,277,25.6,49,[1]


In [34]:
fifa_tables = pd.read_html("https://en.wikipedia.org/wiki/FIFA_World_Cup")

fifa_tables[3].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Ed.,Year,Hosts,NaN,Final,Final,Final,NaN,Third place playoff,Third place playoff,Third place playoff,NaN,No. of teams
1,Ed.,Year,Hosts,NaN,Champions,Score,Runners-up,NaN,Third place,Score,Fourth place,NaN,No. of teams
2,1,1930,Uruguay,NaN,Uruguay,4–2,Argentina,NaN,United States,– [n 1],Yugoslavia,NaN,13
3,2,1934,Italy,NaN,Italy,2–1 (a.e.t.),Czechoslovakia,NaN,Germany,3–2,Austria,NaN,16
4,3,1938,France,NaN,Italy,4–2,Hungary,NaN,Brazil,4–2,Sweden,NaN,15


---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---